<span style="font-size: 25px; font-weight: bold;">Matches - Result(2025)</span>

In [10]:
import pandas as pd
match_result = pd.read_csv("Ipl Matches (2025)/ipl_match_results.csv")

match_result.head()

,No,Match_No,Team1_Team2,Team1_Score,Team2_Score,Result,Date,Time,Venue,Is_Super_Over
0,1,Match 55,Sunrisers Hyderabad vs Delhi Capitals,DC133/7(20.0OV),NaN,No result,"MAY, MON 5 , 7:30 pm IST","MAY, MON 5 , 7:30 pm IST","Rajiv Gandhi International Stadium, Hyderabad",No
1,2,Match 54,Punjab Kings vs Lucknow Super Giants,236/5 (20.0 OV),199/7 (20.0 OV),Punjab Kings Won by 37 Runs,"MAY, SUN 4 , 7:30 pm IST","MAY, SUN 4 , 7:30 pm IST","Himachal Pradesh Cricket Association Stadium, ...",No
2,3,Match 53,KKR vs Rajasthan Royals,206/4 (20.0 OV),205/8 (20.0 OV),Kolkata Knight Riders won,"MAY, SUN 4 , 3:30 pm IST","MAY, SUN 4 , 3:30 pm IST","Eden Gardens, Kolkata",No
3,4,Match 52,RCB vs Chennai Super Kings,213/5 (20.0 OV),211/5 (20.0 OV),Royal Challengers Bengaluru Won by 2 Runs,"MAY, SAT 3 , 7:30 pm IST","MAY, SAT 3 , 7:30 pm IST","M Chinnaswamy Stadium, Bengaluru",No
4,5,Match 51,Gujarat Titans vs Sunrisers Hyderabad,224/6 (20.0 OV),186/6 (20.0 OV),Gujarat Titans Won by 38 Runs,"MAY, FRI 2 , 7:30 pm IST","MAY, FRI 2 , 7:30 pm IST","Narendra Modi Stadium, Ahmedabad",No


In [11]:
match_result.columns = match_result.columns.str.strip()

In [12]:
match_result['Match_Number'] = match_result['Match_No'].str.extract(r'Match (\d+)').astype(int)
match_result = match_result.sort_values(by='Match_Number')
match_result = match_result.drop('Match_Number', axis=1).reset_index(drop=True)

print(match_result)


    No  Match_No                                  Team1_Team2  \
0   55   Match 1                                   KKR vs RCB   
1   54   Match 2      Sunrisers Hyderabad vs Rajasthan Royals   
2   53   Match 3        Chennai Super Kings vs Mumbai Indians   
3   52   Match 4       Delhi Capitals vs Lucknow Super Giants   
4   51   Match 5               Gujarat Titans vs Punjab Kings   
5   50   Match 6                      Rajasthan Royals vs KKR   
6   49   Match 7  Sunrisers Hyderabad vs Lucknow Super Giants   
7   48   Match 8                   Chennai Super Kings vs RCB   
8   47   Match 9             Gujarat Titans vs Mumbai Indians   
9   46  Match 10        Delhi Capitals vs Sunrisers Hyderabad   
10  45  Match 11      Rajasthan Royals vs Chennai Super Kings   
11  44  Match 12                        Mumbai Indians vs KKR   
12  43  Match 13         Lucknow Super Giants vs Punjab Kings   
13  42  Match 14                        RCB vs Gujarat Titans   
14  41  Match 15         

In [13]:
print("Initial DataFrame shape:", match_result.shape)
print("\nColumn names:")
print(match_result.columns.tolist())

Initial DataFrame shape: (55, 10)

Column names:
['No', 'Match_No', 'Team1_Team2', 'Team1_Score', 'Team2_Score', 'Result', 'Date', 'Time', 'Venue', 'Is_Super_Over']


<span style="font-size: 20px; font-weight: bold;">1.1 - Clean the Result Column </span>

In [14]:
def clean_result(result_str):
    if 'Won by' in result_str:
        return result_str
    elif 'Super over' in result_str.lower():
        return 'Delhi Capitals Won by Super Over'
    else:
        return result_str

match_result['Result'] = match_result['Result'].apply(clean_result)

print("\nCleaned Result column:")
print(match_result['Result'].head())



Cleaned Result column:
0    Royal Challengers Bengaluru Won by 7 Wickets
1              Sunrisers Hyderabad Won by 44 Runs
2            Chennai Super Kings Won by 4 Wickets
3                  Delhi Capitals Won by 1 Wicket
4                     Punjab Kings Won by 11 Runs
Name: Result, dtype: object


<span style="font-size: 20px; font-weight: bold;">1.2 - Extract Winner and Winning Margin </span>

In [15]:
import pandas as pd


def extract_winner_margin(result):
    if "Won by" in result:
        parts = result.split("Won by")
        winner = parts[0].strip()
        margin = parts[1].strip()
    elif "won - Super over" in result or "Won - Super over" in result:
        winner = result.split("won")[0].strip().title()  # Title-case to normalize
        margin = "Super Over"
    else:
        winner = None
        margin = None
    return pd.Series([winner, margin])

match_result[['Winner', 'Winning_Margin']] = match_result['Result'].apply(extract_winner_margin)

print(match_result[['No', 'Result', 'Winner', 'Winning_Margin']])



    No                                             Result  \
0   55       Royal Challengers Bengaluru Won by 7 Wickets   
1   54                 Sunrisers Hyderabad Won by 44 Runs   
2   53               Chennai Super Kings Won by 4 Wickets   
3   52                     Delhi Capitals Won by 1 Wicket   
4   51                        Punjab Kings Won by 11 Runs   
5   50             Kolkata Knight Riders Won by 8 Wickets   
6   49              Lucknow Super Giants Won by 5 Wickets   
7   48         Royal Challengers Bengaluru Won by 50 Runs   
8   47                      Gujarat Titans Won by 36 Runs   
9   46                    Delhi Capitals Won by 7 Wickets   
10  45                     Rajasthan Royals Won by 6 Runs   
11  44                    Mumbai Indians Won by 8 Wickets   
12  43                      Punjab Kings Won by 8 Wickets   
13  42                    Gujarat Titans Won by 8 Wickets   
14  41               Kolkata Knight Riders Won by 80 Runs   
15  40                Lu

<span style="font-size: 20px; font-weight: bold;">1.3 - Extract Team1 and Team2 names separately </span>

In [16]:
def extract_teams(team_str):
    if ' vs ' in team_str:
        return team_str.split(' vs ')
    return None, None

match_result[['Team1', 'Team2']] = pd.DataFrame(match_result['Team1_Team2'].apply(extract_teams).tolist(), index=match_result.index)

print("\nSeparated team columns:")
print(match_result[['Team1', 'Team2']].head())



Separated team columns:
                 Team1                 Team2
0                  KKR                   RCB
1  Sunrisers Hyderabad      Rajasthan Royals
2  Chennai Super Kings        Mumbai Indians
3       Delhi Capitals  Lucknow Super Giants
4       Gujarat Titans          Punjab Kings


<span style="font-size: 20px; font-weight: bold;">1.4 - Clean Date Column </span>

In [18]:
import re
from datetime import datetime

def clean_date(date_str):

    date_match = re.search(r'([A-Z]{3}, [A-Z]{3} \d{1,2})', date_str)
    if date_match:
        date_part = date_match.group(1)

        date_with_year = date_part + ', 2025'

        try:
            return datetime.strptime(date_with_year, '%b, %a %d, %Y')
        except ValueError:
            return None
    return None

match_result['Date'] = match_result['Date'].apply(clean_date)

# Check the cleaned dates
print("\nCleaned date column:")
print(match_result['Date'].head())



Cleaned date column:
0   2025-03-22
1   2025-03-23
2   2025-03-23
3   2025-03-24
4   2025-03-25
Name: Date, dtype: datetime64[ns]


<span style="font-size: 20px; font-weight: bold;">1.5 - Clean Time Column </span>

In [19]:
import re
from datetime import datetime

def clean_time(time_str):
    time_match = re.search(r'(\d{1,2}:\d{2}\s*[AP]M)', time_str, re.IGNORECASE)
    
    if time_match:
        time_part = time_match.group(1).upper().strip()
        try:
            return datetime.strptime(time_part, '%I:%M %p').time()
        except ValueError:
            return None
    
    return None

# Apply it to your DataFrame
match_result['Time'] = match_result['Time'].apply(clean_time)

# Optional: Check cleaned times
print("\nCleaned time column:")
print(match_result['Time'].head())


Cleaned time column:
0    19:30:00
1    15:30:00
2    19:30:00
3    19:30:00
4    19:30:00
Name: Time, dtype: object


<span style="font-size: 20px; font-weight: bold;">1.6 - Extract Score Details for Team 1 </span>

In [20]:
def extract_score(score_str):
    
    if isinstance(score_str, str):
       
        runs_match = re.search(r'(\d+)(?:/\d+)?', score_str)
        runs = int(runs_match.group(1)) if runs_match else None
        
        
        wickets_match = re.search(r'/(\d+)', score_str)
        wickets = int(wickets_match.group(1)) if wickets_match else 10 if runs is not None else None
        
        
        overs_match = re.search(r'\((\d+\.\d+|\d+) OV\)', score_str)
        overs = float(overs_match.group(1)) if overs_match else None
        
        return runs, wickets, overs
    return None, None, None


match_result['Team1_Runs'], match_result['Team1_Wickets'], match_result['Team1_Overs'] = zip(*match_result['Team1_Score'].apply(extract_score))

print("\nTeam1 score details:")
print(match_result[['Team1_Runs', 'Team1_Wickets', 'Team1_Overs']].head())


Team1 score details:
   Team1_Runs  Team1_Wickets  Team1_Overs
0         174              8         20.0
1         286              6         20.0
2         158              6         19.1
3         211              9         19.3
4         232              5         20.0


<span style="font-size: 20px; font-weight: bold;">1.7 - Extract Score Details for Team 2 </span> 

In [21]:
# Apply to Team2 Score
match_result['Team2_Runs'], match_result['Team2_Wickets'], match_result['Team2_Overs'] = zip(*match_result['Team2_Score'].apply(extract_score))

# Check Team2 score details
print("\nTeam2 score details:")
print(match_result[['Team2_Runs', 'Team2_Wickets', 'Team2_Overs']].head())



Team2 score details:
   Team2_Runs  Team2_Wickets  Team2_Overs
0       177.0            3.0         16.2
1       242.0            6.0         20.0
2       155.0            9.0         20.0
3       209.0            8.0         20.0
4       243.0            5.0         20.0


<span style="font-size: 20px; font-weight: bold;">1.8 - Clean Venue and Super Over Information </span>  

In [22]:
def clean_venue(venue_str):
    if isinstance(venue_str, str):
        return venue_str.strip()
    return None

match_result['Venue'] = match_result['Venue'].apply(clean_venue)


print(match_result[['Venue', 'Is_Super_Over']].head())

                                               Venue Is_Super_Over
0                              Eden Gardens, Kolkata            No
1      Rajiv Gandhi International Stadium, Hyderabad            No
2                    MA Chidambaram Stadium, Chennai            No
3  Dr YS Rajasekhara Reddy ACA-VDCA Cricket Stadi...            No
4                   Narendra Modi Stadium, Ahmedabad            No


In [23]:
print(match_result.columns.tolist())

['No', 'Match_No', 'Team1_Team2', 'Team1_Score', 'Team2_Score', 'Result', 'Date', 'Time', 'Venue', 'Is_Super_Over', 'Winner', 'Winning_Margin', 'Team1', 'Team2', 'Team1_Runs', 'Team1_Wickets', 'Team1_Overs', 'Team2_Runs', 'Team2_Wickets', 'Team2_Overs']


<span style="font-size: 20px; font-weight: bold;">1.8 - Restructure the DataFrame </span> 

In [24]:
columns_to_drop = ['Team1_Team2', 'Team1_Score', 'Team2_Score', 'Result']
existing_columns_to_drop = [col for col in columns_to_drop if col in match_result.columns]

match_result = match_result.drop(columns=existing_columns_to_drop)

# Reorder columns for better readability (fixed the typo in column name)
column_order = ['Match_No', 'Date', 'Time', 'Team1', 'Team2', 'Team1_Runs', 'Team1_Wickets', 'Team1_Overs',
                'Team2_Runs', 'Team2_Wickets', 'Team2_Overs', 'Winner', 'Winning_Margin',
                'Venue', 'Is_Super_Over']
match_result = match_result[column_order]

# Check for missing values
print("\nMissing values in each column:")
print(match_result.isnull().sum())


Missing values in each column:
Match_No          0
Date              0
Time              0
Team1             0
Team2             0
Team1_Runs        0
Team1_Wickets     0
Team1_Overs       1
Team2_Runs        1
Team2_Wickets     1
Team2_Overs       1
Winner            3
Winning_Margin    3
Venue             0
Is_Super_Over     0
dtype: int64


<span style="font-size: 20px; font-weight: bold;">1.9 - Standardizing Team Names </span>   

In [25]:
team_name_map = {
    'KKR': 'Kolkata Knight Riders',
    'RCB': 'Royal Challengers Bangalore',
    'GT': 'Gujarat Titans',
    'DC': 'Delhi Capitals',
    'MI': 'Mumbai Indians',
    'CSK': 'Chennai Super Kings',
    'PBKS': 'Punjab Kings',
    'LSG': 'Lucknow Super Giants',
    'SRH': 'Sunrisers Hyderabad'
}
# Replace short names with full names in Team1 and Team2
match_result['Team1'] = match_result['Team1'].replace(team_name_map)
match_result['Team2'] = match_result['Team2'].replace(team_name_map)


<span style="font-size: 20px; font-weight: bold;">2.0 - Save Data </span>    

In [27]:
import os

Folder_name = 'Ipl Matches (2025)'
os.makedirs(Folder_name, exist_ok=True)  # Create folder if it doesn't exist

file_path = os.path.join(Folder_name, 'cleaned_match_results.csv')
match_result.to_csv(file_path, index=False)

print("\nCleaned data saved to:", file_path)



Cleaned data saved to: Ipl Matches (2025)\cleaned_match_results.csv


<span style="font-size: 25px; font-weight: bold;">Matches - Fixtures(2025)</span>

<span style="font-size: 20px; font-weight: bold;">2.1 Load the IPL matches fixture data</span> 

In [28]:
try:
    match_fixtures = pd.read_csv("Ipl Matches (2025)/ipl_matches_Fixtures.csv")
    print("Successfully loaded ipl_matches_Fixtures.csv")
    print("Initial DataFrame shape:", match_fixtures.shape)
    print("\nFirst 5 rows:")
    print(match_fixtures.head())
except Exception as e:
    print(f"Error loading file: {e}")

Successfully loaded ipl_matches_Fixtures.csv
Initial DataFrame shape: (19, 6)

First 5 rows:
   No  Match_No                       Team1_Team2         Date         Time  \
0   1  Match 56  Mumbai Indians vs Gujarat Titans   MAY, TUE 6  7:30 pm IST   
1   2  Match 57        KKR vs Chennai Super Kings   MAY, WED 7  7:30 pm IST   
2   3  Match 58    Punjab Kings vs Delhi Capitals   MAY, THU 8  7:30 pm IST   
3   4  Match 59       Lucknow Super Giants vs RCB   MAY, FRI 9  7:30 pm IST   
4   5  Match 60        Sunrisers Hyderabad vs KKR  MAY, SAT 10  7:30 pm IST   

                                               Venue  
0                           Wankhede Stadium, Mumbai  
1                              Eden Gardens, Kolkata  
2  Himachal Pradesh Cricket Association Stadium, ...  
3  Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...  
4      Rajiv Gandhi International Stadium, Hyderabad  


<span style="font-size: 20px; font-weight: bold;">2.2 - Restructure the DataFrame </span> 

In [29]:
# Clean column names
match_fixtures.columns = match_fixtures.columns.str.strip()
print("\nColumn names after cleaning:")
print(match_fixtures.columns.tolist())

# Check for missing values
print("\nMissing values in each column:")
print(match_fixtures.isnull().sum())

# Extract match number
match_fixtures['Match_Number'] = match_fixtures['Match_No'].str.extract(r'Match (\d+)').astype(int)
match_fixtures = match_fixtures.sort_values(by='Match_Number')
match_fixtures = match_fixtures.drop('Match_Number', axis=1).reset_index(drop=True)


Column names after cleaning:
['No', 'Match_No', 'Team1_Team2', 'Date', 'Time', 'Venue']

Missing values in each column:
No             0
Match_No       0
Team1_Team2    0
Date           0
Time           0
Venue          0
dtype: int64


<span style="font-size: 20px; font-weight: bold;">2.3 - Split Team1_Team2 into separate columns </span> 

In [30]:
def extract_teams(team_str):
    if ' vs ' in team_str:
        teams = team_str.split(' vs ')
        return pd.Series(teams)
    return pd.Series([None, None])

match_fixtures[['Team1', 'Team2']] = match_fixtures['Team1_Team2'].apply(extract_teams)
match_fixtures

,No,Match_No,Team1_Team2,Date,Time,Venue,Team1,Team2
0,1,Match 56,Mumbai Indians vs Gujarat Titans,"MAY, TUE 6",7:30 pm IST,"Wankhede Stadium, Mumbai",Mumbai Indians,Gujarat Titans
1,2,Match 57,KKR vs Chennai Super Kings,"MAY, WED 7",7:30 pm IST,"Eden Gardens, Kolkata",KKR,Chennai Super Kings
2,3,Match 58,Punjab Kings vs Delhi Capitals,"MAY, THU 8",7:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Punjab Kings,Delhi Capitals
3,4,Match 59,Lucknow Super Giants vs RCB,"MAY, FRI 9",7:30 pm IST,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Lucknow Super Giants,RCB
4,5,Match 60,Sunrisers Hyderabad vs KKR,"MAY, SAT 10",7:30 pm IST,"Rajiv Gandhi International Stadium, Hyderabad",Sunrisers Hyderabad,KKR
5,6,Match 61,Punjab Kings vs Mumbai Indians,"MAY, SUN 11",3:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Punjab Kings,Mumbai Indians
6,7,Match 62,Delhi Capitals vs Gujarat Titans,"MAY, SUN 11",7:30 pm IST,"Arun Jaitley Stadium, Delhi",Delhi Capitals,Gujarat Titans
7,8,Match 63,Chennai Super Kings vs Rajasthan Royals,"MAY, MON 12",7:30 pm IST,"MA Chidambaram Stadium, Chennai",Chennai Super Kings,Rajasthan Royals
8,9,Match 64,RCB vs Sunrisers Hyderabad,"MAY, TUE 13",7:30 pm IST,"M Chinnaswamy Stadium, Bengaluru",RCB,Sunrisers Hyderabad
9,10,Match 65,Gujarat Titans vs Lucknow Super Giants,"MAY, WED 14",7:30 pm IST,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,Lucknow Super Giants


<span style="font-size: 20px; font-weight: bold;">2.4 - Clean Date Column </span>  

In [31]:
import re
from datetime import datetime

def clean_date(date_str):
    date_fixtures = re.search(r'([A-Z]{3}, [A-Z]{3} \d{1,2})', date_str)
    if date_fixtures:
        date_part = date_fixtures.group(1)
        date_with_year = date_part + ', 2025'
        try:
            return datetime.strptime(date_with_year, '%b, %a %d, %Y')
        except ValueError:
            return None
    return None

match_fixtures['Date'] = match_fixtures['Date'].apply(clean_date)

print("\nCleaned date column:")
print(match_fixtures['Date'].head())



Cleaned date column:
0   2025-05-06
1   2025-05-07
2   2025-05-08
3   2025-05-09
4   2025-05-10
Name: Date, dtype: datetime64[ns]


<span style="font-size: 20px; font-weight: bold;">2.5 - Clean time field </span>  

In [33]:
def clean_time(time_str):
    if not isinstance(time_str, str):
        return None
    
    time_match = re.search(r'(\d{1,2}:\d{2})\s*([ap]m)', time_str.lower())
    if time_match:
        time_part = time_match.group(1)
        am_pm = time_match.group(2)
        
        hour, minute = map(int, time_part.split(':'))
        
        # Convert to 24-hour format
        if am_pm == 'pm' and hour < 12:
            hour += 12
        elif am_pm == 'am' and hour == 12:
            hour = 0
            
        return f"{hour:02d}:{minute:02d}"
    
    return None

match_fixtures['Time_Formatted'] = match_fixtures['Time'].apply(clean_time)

In [34]:
print(match_result.columns.tolist())

['Match_No', 'Date', 'Time', 'Team1', 'Team2', 'Team1_Runs', 'Team1_Wickets', 'Team1_Overs', 'Team2_Runs', 'Team2_Wickets', 'Team2_Overs', 'Winner', 'Winning_Margin', 'Venue', 'Is_Super_Over']


<span style="font-size: 20px; font-weight: bold;">2.6 - cleaned DataFrame </span>   

In [35]:
match_fixtures['Status'] = 'Scheduled'

match_fixtures[['Team1', 'Team2']] = match_fixtures['Team1_Team2'].str.split(' vs ', expand=True)

# Create the final cleaned DataFrame
cleaned_results = match_fixtures[[
    'No', 'Match_No', 'Team1', 'Team2',
     'Date', 'Time', 'Venue', 'Status'
]]

cleaned_results

,No,Match_No,Team1,Team2,Date,Time,Venue,Status
0,1,Match 56,Mumbai Indians,Gujarat Titans,2025-05-06,7:30 pm IST,"Wankhede Stadium, Mumbai",Scheduled
1,2,Match 57,KKR,Chennai Super Kings,2025-05-07,7:30 pm IST,"Eden Gardens, Kolkata",Scheduled
2,3,Match 58,Punjab Kings,Delhi Capitals,2025-05-08,7:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Scheduled
3,4,Match 59,Lucknow Super Giants,RCB,2025-05-09,7:30 pm IST,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Scheduled
4,5,Match 60,Sunrisers Hyderabad,KKR,2025-05-10,7:30 pm IST,"Rajiv Gandhi International Stadium, Hyderabad",Scheduled
5,6,Match 61,Punjab Kings,Mumbai Indians,2025-05-11,3:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Scheduled
6,7,Match 62,Delhi Capitals,Gujarat Titans,2025-05-11,7:30 pm IST,"Arun Jaitley Stadium, Delhi",Scheduled
7,8,Match 63,Chennai Super Kings,Rajasthan Royals,2025-05-12,7:30 pm IST,"MA Chidambaram Stadium, Chennai",Scheduled
8,9,Match 64,RCB,Sunrisers Hyderabad,2025-05-13,7:30 pm IST,"M Chinnaswamy Stadium, Bengaluru",Scheduled
9,10,Match 65,Gujarat Titans,Lucknow Super Giants,2025-05-14,7:30 pm IST,"Narendra Modi Stadium, Ahmedabad",Scheduled


In [36]:
print(match_fixtures.columns)

Index(['No', 'Match_No', 'Team1_Team2', 'Date', 'Time', 'Venue', 'Team1',
       'Team2', 'Time_Formatted', 'Status'],
      dtype='object')


<span style="font-size: 20px; font-weight: bold;">2.7 - Standardizing IPL Fixture Data  </span>   

In [37]:
cleaned_results['Team1'] = cleaned_results['Team1'].apply(lambda x: 'TBD' if x == 'TBD' else x)
cleaned_results['Team2'] = cleaned_results['Team2'].apply(lambda x: 'TBD' if x == 'TBD' else x)

team_name_map = {
    'KKR': 'Kolkata Knight Riders',
    'RCB': 'Royal Challengers Bangalore',
    'GT': 'Gujarat Titans',
    'DC': 'Delhi Capitals',
    'MI': 'Mumbai Indians',
    'CSK': 'Chennai Super Kings',
    'PBKS': 'Punjab Kings',
    'LSG': 'Lucknow Super Giants',
    'SRH': 'Sunrisers Hyderabad',
    'RR': 'Rajasthan Royals'
}

# Replace short names with full names
cleaned_results['Team1'] = cleaned_results['Team1'].replace(team_name_map)
cleaned_results['Team2'] = cleaned_results['Team2'].replace(team_name_map)

# Clean venue names
def clean_venue(venue_str):
    if isinstance(venue_str, str):
        return venue_str.strip()
    return None


cleaned_results

C:\Users\ajmal\AppData\Local\Temp\ipykernel_46064\1725719772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_results['Team1'] = cleaned_results['Team1'].apply(lambda x: 'TBD' if x == 'TBD' else x)
C:\Users\ajmal\AppData\Local\Temp\ipykernel_46064\1725719772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_results['Team2'] = cleaned_results['Team2'].apply(lambda x: 'TBD' if x == 'TBD' else x)
C:\Users\ajmal\AppData\Local\Temp\ipykernel_46064\1725719772.py:18: SettingWithCopyWarning: 
A

,No,Match_No,Team1,Team2,Date,Time,Venue,Status
0,1,Match 56,Mumbai Indians,Gujarat Titans,2025-05-06,7:30 pm IST,"Wankhede Stadium, Mumbai",Scheduled
1,2,Match 57,Kolkata Knight Riders,Chennai Super Kings,2025-05-07,7:30 pm IST,"Eden Gardens, Kolkata",Scheduled
2,3,Match 58,Punjab Kings,Delhi Capitals,2025-05-08,7:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Scheduled
3,4,Match 59,Lucknow Super Giants,Royal Challengers Bangalore,2025-05-09,7:30 pm IST,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Scheduled
4,5,Match 60,Sunrisers Hyderabad,Kolkata Knight Riders,2025-05-10,7:30 pm IST,"Rajiv Gandhi International Stadium, Hyderabad",Scheduled
5,6,Match 61,Punjab Kings,Mumbai Indians,2025-05-11,3:30 pm IST,"Himachal Pradesh Cricket Association Stadium, ...",Scheduled
6,7,Match 62,Delhi Capitals,Gujarat Titans,2025-05-11,7:30 pm IST,"Arun Jaitley Stadium, Delhi",Scheduled
7,8,Match 63,Chennai Super Kings,Rajasthan Royals,2025-05-12,7:30 pm IST,"MA Chidambaram Stadium, Chennai",Scheduled
8,9,Match 64,Royal Challengers Bangalore,Sunrisers Hyderabad,2025-05-13,7:30 pm IST,"M Chinnaswamy Stadium, Bengaluru",Scheduled
9,10,Match 65,Gujarat Titans,Lucknow Super Giants,2025-05-14,7:30 pm IST,"Narendra Modi Stadium, Ahmedabad",Scheduled


In [38]:
print(match_fixtures.columns.tolist())

['No', 'Match_No', 'Team1_Team2', 'Date', 'Time', 'Venue', 'Team1', 'Team2', 'Time_Formatted', 'Status']


<span style="font-size: 20px; font-weight: bold;">2.8 - cleaned DataFrame </span>   

In [39]:
# Check for missing values
print("\nMissing values in each column:")
print(cleaned_results.isnull().sum())



Missing values in each column:
No          0
Match_No    0
Team1       0
Team2       0
Date        0
Time        0
Venue       0
Status      0
dtype: int64


<span style="font-size: 20px; font-weight: bold;">2.9 - Save Data </span>    

In [41]:
# Display final cleaned DataFrame
print("\nFinal cleaned DataFrame:")
print(cleaned_results.head())
print("\nDataFrame shape:", cleaned_results.shape)

# Save the cleaned data
folder_name = 'Ipl Matches (2025)'
os.makedirs(folder_name, exist_ok=True)

file_path = os.path.join(folder_name, 'cleaned_ipl_fixtures.csv')
cleaned_results.to_csv(file_path, index=False)

print(f"\nCleaned fixture data saved to: {file_path}")


Final cleaned DataFrame:
   No  Match_No                  Team1                        Team2  \
0   1  Match 56         Mumbai Indians               Gujarat Titans   
1   2  Match 57  Kolkata Knight Riders          Chennai Super Kings   
2   3  Match 58           Punjab Kings               Delhi Capitals   
3   4  Match 59   Lucknow Super Giants  Royal Challengers Bangalore   
4   5  Match 60    Sunrisers Hyderabad        Kolkata Knight Riders   

        Date         Time                                              Venue  \
0 2025-05-06  7:30 pm IST                           Wankhede Stadium, Mumbai   
1 2025-05-07  7:30 pm IST                              Eden Gardens, Kolkata   
2 2025-05-08  7:30 pm IST  Himachal Pradesh Cricket Association Stadium, ...   
3 2025-05-09  7:30 pm IST  Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...   
4 2025-05-10  7:30 pm IST      Rajiv Gandhi International Stadium, Hyderabad   

      Status  
0  Scheduled  
1  Scheduled  
2  Scheduled  
3  Sch

<span style="font-size: 25px; font-weight: bold;">IPL Points Tables - (2008 - 2025)</span>

In [42]:
import pandas as pd
import os
import glob
import re

# Define the directory containing the CSV files
data_dir = "IPL Points Tables"

# Check if directory exists
if not os.path.exists(data_dir):
    print(f"Directory '{data_dir}' does not exist. Creating it...")
    os.makedirs(data_dir)
    print("Please place your IPL points table CSV files in this directory.")
    exit()

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(data_dir, "IPL_Points_Table_*.csv"))

if len(csv_files) == 0:
    print(f"No CSV files found in '{data_dir}'. Please add your IPL points table CSV files.")
    exit()

# Function to extract year from filename
def extract_year(filename):
    match = re.search(r'(\d{4})', filename)
    if match:
        return int(match.group(1))
    return None

# List to store individual dataframes
all_dfs = []

# Read each CSV file and add a year column
for file in sorted(csv_files, key=extract_year):
    year = extract_year(file)
    if year:
        try:
            df = pd.read_csv(file)
            
            # Add year column
            df['Season'] = year
            
            # Standardize column names (if they vary between files)
            if 'TEAM' in df.columns:
                df.rename(columns={'TEAM': 'Team'}, inplace=True)
            if 'P' in df.columns:
                df.rename(columns={'P': 'Matches'}, inplace=True)
            if 'W' in df.columns:
                df.rename(columns={'W': 'Wins'}, inplace=True)
            if 'L' in df.columns:
                df.rename(columns={'L': 'Losses'}, inplace=True)
            if 'NR' in df.columns:
                df.rename(columns={'NR': 'No_Result'}, inplace=True)
            if 'NRR' in df.columns:
                df.rename(columns={'NRR': 'Net_Run_Rate'}, inplace=True)
            if 'PTS' in df.columns:
                df.rename(columns={'PTS': 'Points'}, inplace=True)
                
            # Clean team names (if needed)
            if 'Team' in df.columns:
                # Remove any special characters or trailing spaces
                df['Team'] = df['Team'].str.strip()
            
            # Append to list
            all_dfs.append(df)
            print(f"Processed {file} - Year: {year}")
        except Exception as e:
            print(f"Error processing {file}: {e}")
    else:
        print(f"Could not extract year from {file}. Skipping...")

# Check if we have data to combine
if len(all_dfs) == 0:
    print("No data was successfully loaded. Please check your CSV files.")
    exit()

# Combine all dataframes
combined_df = pd.concat(all_dfs, ignore_index=True)

# Sort by year and then by position
combined_df = combined_df.sort_values(by=['Season', 'POS'])

# Additional cleaning
# Convert numeric columns to appropriate types
numeric_cols = ['Matches', 'Wins', 'Losses', 'No_Result', 'Net_Run_Rate', 'Points']
for col in numeric_cols:
    if col in combined_df.columns:
        try:
            if col == 'Net_Run_Rate':
                combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
            else:
                combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce').fillna(0).astype(int)
        except Exception as e:
            print(f"Error converting {col} to numeric: {e}")

# Standardize team names (mapping variations of the same team)
team_mapping = {
    'DD': 'Delhi Daredevils',
    'DC': 'Delhi Capitals',  # Renamed from Delhi Daredevils
    'PBKS': 'Punjab Kings',  # Renamed from Kings XI Punjab
    'KXIP': 'Punjab Kings',
    'CSK': 'Chennai Super Kings',
    'MI': 'Mumbai Indians',
    'KKR': 'Kolkata Knight Riders',
    'RCB': 'Royal Challengers Bangalore',
    'RR': 'Rajasthan Royals',
    'SRH': 'Sunrisers Hyderabad',
    'DEC': 'Deccan Chargers',
    'GL': 'Gujarat Lions',
    'RPS': 'Rising Pune Supergiant',
    'PWI': 'Pune Warriors India',
    'KTK': 'Kochi Tuskers Kerala',
    'GT': 'Gujarat Titans',
    'LSG': 'Lucknow Super Giants'
}

# Apply team name standardization if Team column exists
if 'Team' in combined_df.columns:
    combined_df['Team'] = combined_df['Team'].map(lambda x: team_mapping.get(x, x))

# Save the combined dataset
output_dir = 'Ipl Matches (2025)'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "IPL - Points Table - 2008 to 2025.csv")
combined_df.to_csv(output_path, index=False)

print(f"Fixtures data saved to: {output_path}")

# Print stats about the combined dataset
print("\nDataset Statistics:")
print(f"Total records: {len(combined_df)}")
print(f"Years covered: {combined_df['Season'].min()} to {combined_df['Season'].max()}")
print(f"Teams included: {combined_df['Team'].nunique()}")

# Basic analysis
if 'Team' in combined_df.columns and 'Wins' in combined_df.columns:
    team_wins = combined_df.groupby('Team')['Wins'].sum().sort_values(ascending=False)
    print("\nTotal Wins by Team:")
    print(team_wins)

if 'Team' in combined_df.columns and 'Points' in combined_df.columns:
    team_points = combined_df.groupby('Team')['Points'].sum().sort_values(ascending=False)
    print("\nTotal Points by Team:")
    print(team_points)


Directory 'IPL Points Tables' does not exist. Creating it...
Please place your IPL points table CSV files in this directory.
No CSV files found in 'IPL Points Tables'. Please add your IPL points table CSV files.
No data was successfully loaded. Please check your CSV files.


ValueError: No objects to concatenate

<span style="font-size: 25px; font-weight: bold;">Ipl Stat 2025 - Batting & Bowling </span> 

In [350]:
import os
import pandas as pd
import numpy as np

# Define the directory structure
base_dir = "Ipl Stat 2025"
batting_dir = os.path.join(base_dir, "batting")
bowling_dir = os.path.join(base_dir, "bowling")

# Step 3.1: Load & Clean Batting Stats
def load_clean_batting_stats():
    print("Loading and cleaning batting statistics...")
    
    # Define batting files
    batting_files = {
        "best_batting_averages": os.path.join(batting_dir, "best_batting_averages.csv"),
        "most_centuries": os.path.join(batting_dir, "most_centuries.csv"),
        "most_fifties": os.path.join(batting_dir, "most_fifties.csv"),
        "orange_cap": os.path.join(batting_dir, "orange_cap.csv")
    }
    
    batting_dfs = {}
    
    # Load each batting file
    for name, file_path in batting_files.items():
        try:
            # Read CSV file
            df = pd.read_csv(file_path)
            
            # Clean the data
            
            # 1. Handle player name and team separately
            if 'PLAYER' in df.columns:
                # Split the PLAYER column into PLAYER and TEAM
                df[['PLAYER', 'TEAM']] = df['PLAYER'].str.split('\n', expand=True)
                
            # 2. Convert numeric columns to appropriate types
            numeric_cols = ['MAT', 'INNS', 'NO', 'RUNS', 'AVG', 'BF', 'SR', '100', '50', '4S', '6S']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 3. Clean the highest score column
            if 'HS' in df.columns:
                # Extract numeric value and not out indicator
                df['HS_VALUE'] = df['HS'].str.extract(r'(\d+)', expand=False).astype(float)
                df['HS_NOTOUT'] = df['HS'].str.contains(r'\*', na=False).astype(int)
            
            # Store the cleaned dataframe
            batting_dfs[name] = df
            print(f"Successfully loaded and cleaned {name} data")
            
        except Exception as e:
            print(f"Error loading {name}: {e}")
    
    return batting_dfs

# Step 3.2: Load & Clean Bowling Stats
def load_clean_bowling_stats():
    print("Loading and cleaning bowling statistics...")
    
    # Define bowling files
    bowling_files = {
        "best_bowling_figures": os.path.join(bowling_dir, "best_bowling_figures.csv"),
        "purple_cap": os.path.join(bowling_dir, "purple_cap.csv"),
        "tata_ipl_green_dot_balls": os.path.join(bowling_dir, "tata_ipl_green_dot_balls.csv")
    }
    
    bowling_dfs = {}
    
    # Load each bowling file
    for name, file_path in bowling_files.items():
        try:
            # Read CSV file
            df = pd.read_csv(file_path)
            
            # Clean the data
            
            # 1. Handle player name and team separately
            if 'PLAYER' in df.columns:
                # Split the PLAYER column into PLAYER and TEAM
                df[['PLAYER', 'TEAM']] = df['PLAYER'].str.split('\n', expand=True)
            
            # 2. Convert numeric columns to appropriate types
            numeric_cols = ['WKTS', 'MAT', 'INNS', 'RUNS', 'AVG', 'ECON', 'SR', '4W', '5W', 'DOTS']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 3. Process BBI (Best Bowling in an Innings) column
            if 'BBI' in df.columns:
                # Split BBI into wickets and runs
                df[['BBI_RUNS', 'BBI_WKTS']] = df['BBI'].str.split('/', expand=True)
                df['BBI_RUNS'] = pd.to_numeric(df['BBI_RUNS'], errors='coerce')
                df['BBI_WKTS'] = pd.to_numeric(df['BBI_WKTS'], errors='coerce')
            
            # 4. Handle overs column (which can have decimal values)
            if 'OV' in df.columns:
                # Convert overs to balls (1 over = 6 balls)
                df['OV'] = pd.to_numeric(df['OV'], errors='coerce')
                df['BALLS'] = (df['OV'].astype(float).apply(lambda x: int(x)) * 6 + 
                             (df['OV'] - df['OV'].astype(int)) * 10).astype(int)
            
            # Store the cleaned dataframe
            bowling_dfs[name] = df
            print(f"Successfully loaded and cleaned {name} data")
            
        except Exception as e:
            print(f"Error loading {name}: {e}")
    
    return bowling_dfs

# Additional preprocessing functions

def calculate_batting_metrics(batting_dfs):
    """Calculate additional batting metrics based on the data"""
    
    for name, df in batting_dfs.items():
        # Calculate strike rate if not present but we have RUNS and BF
        if 'SR' not in df.columns and 'RUNS' in df.columns and 'BF' in df.columns:
            df['SR'] = (df['RUNS'] / df['BF'] * 100).round(2)
        
        # Calculate average if not present but we have RUNS, INNS, and NO
        if 'AVG' not in df.columns and all(col in df.columns for col in ['RUNS', 'INNS', 'NO']):
            df['AVG'] = (df['RUNS'] / (df['INNS'] - df['NO'])).round(2)
        
        # Calculate boundary percentage
        if all(col in df.columns for col in ['4S', '6S', 'BF']):
            df['BOUNDARY_PCT'] = ((df['4S'] + df['6S']) / df['BF'] * 100).round(2)
            
        # Calculate runs from boundaries
        if all(col in df.columns for col in ['4S', '6S']):
            df['BOUNDARY_RUNS'] = df['4S'] * 4 + df['6S'] * 6
            df['BOUNDARY_RUNS_PCT'] = (df['BOUNDARY_RUNS'] / df['RUNS'] * 100).round(2)
            
        batting_dfs[name] = df
        
    return batting_dfs

def calculate_bowling_metrics(bowling_dfs):
    """Calculate additional bowling metrics based on the data"""
    
    for name, df in bowling_dfs.items():
        # Calculate economy rate if not present but we have RUNS and OV
        if 'ECON' not in df.columns and 'RUNS' in df.columns and 'OV' in df.columns:
            df['ECON'] = (df['RUNS'] / df['OV']).round(2)
        
        # Calculate strike rate if not present but we have BALLS and WKTS
        if 'SR' not in df.columns and 'BALLS' in df.columns and 'WKTS' in df.columns:
            df['SR'] = (df['BALLS'] / df['WKTS']).round(2)
        
        # Calculate average if not present but we have RUNS and WKTS
        if 'AVG' not in df.columns and 'RUNS' in df.columns and 'WKTS' in df.columns:
            df['AVG'] = (df['RUNS'] / df['WKTS']).round(2)
            
        # Calculate dot ball percentage if we have DOTS and BALLS
        if 'DOTS' in df.columns and 'BALLS' in df.columns:
            df['DOT_BALL_PCT'] = (df['DOTS'] / df['BALLS'] * 100).round(2)
            
        bowling_dfs[name] = df
        
    return bowling_dfs

def combine_player_stats(batting_dfs, bowling_dfs):
    """Combine stats to create comprehensive player profiles"""
    
    # Extract main batting and bowling stats
    batting_main = batting_dfs.get('orange_cap', None)
    bowling_main = bowling_dfs.get('purple_cap', None)
    
    if batting_main is None or bowling_main is None:
        print("Missing required datasets to create player profiles")
        return None
    
    # Create a set of all players
    all_players = set(batting_main['PLAYER']).union(set(bowling_main['PLAYER']))
    
    # Initialize player profiles DataFrame
    player_profiles = pd.DataFrame({'PLAYER': list(all_players)})
    
    # Add team information
    player_team_map = {}
    
    # Extract from batting data
    for _, row in batting_main.iterrows():
        if 'TEAM' in row and pd.notna(row['TEAM']):
            player_team_map[row['PLAYER']] = row['TEAM']
    
    # Extract from bowling data (if not already in map)
    for _, row in bowling_main.iterrows():
        if row['PLAYER'] not in player_team_map and 'TEAM' in row and pd.notna(row['TEAM']):
            player_team_map[row['PLAYER']] = row['TEAM']
    
    # Add team to profiles
    player_profiles['TEAM'] = player_profiles['PLAYER'].map(player_team_map)
    
    # Add batting stats
    player_profiles = player_profiles.merge(
        batting_main[['PLAYER', 'MAT', 'INNS', 'RUNS', 'AVG', 'SR', '100', '50', '4S', '6S']], 
        on='PLAYER', how='left', suffixes=('', '_BAT')
    )
    
    # Add bowling stats
    player_profiles = player_profiles.merge(
        bowling_main[['PLAYER', 'WKTS', 'ECON', 'AVG', 'SR', '4W', '5W']], 
        on='PLAYER', how='left', suffixes=('', '_BOWL')
    )
    
    return player_profiles

# Main execution
def main():
    # Create directories if they don't exist
    os.makedirs(batting_dir, exist_ok=True)
    os.makedirs(bowling_dir, exist_ok=True)
    
    # Process batting statistics
    batting_dfs = load_clean_batting_stats()
    batting_dfs = calculate_batting_metrics(batting_dfs)
    
    # Process bowling statistics
    bowling_dfs = load_clean_bowling_stats()
    bowling_dfs = calculate_bowling_metrics(bowling_dfs)
    
    # Create player profiles
    player_profiles = combine_player_stats(batting_dfs, bowling_dfs)
    
    # Output cleaned data to CSV files
    print("\nSaving cleaned data to CSV files...")
    
    for name, df in batting_dfs.items():
        print(f"\n{name}:\n{df.isnull().sum()}")
        output_path = os.path.join(batting_dir, f"{name}_cleaned.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved {output_path}")
    
    for name, df in bowling_dfs.items():
        print(f"\n{name}:\n{df.isnull().sum()}")
        output_path = os.path.join(bowling_dir, f"{name}_cleaned.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved {output_path}")
    
    if player_profiles is not None:
        output_path = os.path.join(base_dir, "player_profiles.csv")
        player_profiles.to_csv(output_path, index=False)
        print(f"Saved {output_path}")
    
    print("\nData processing complete!")


if __name__ == "__main__":
    main()

Loading and cleaning batting statistics...
Successfully loaded and cleaned best_batting_averages data
Successfully loaded and cleaned most_centuries data
Successfully loaded and cleaned most_fifties data
Successfully loaded and cleaned orange_cap data
Loading and cleaning bowling statistics...
Successfully loaded and cleaned best_bowling_figures data
Successfully loaded and cleaned purple_cap data
Successfully loaded and cleaned tata_ipl_green_dot_balls data

Saving cleaned data to CSV files...

best_batting_averages:
POS                  0
PLAYER               0
AVG                  0
MAT                  0
INNS                 0
NO                   0
RUNS                 0
HS                   0
BF                   0
SR                   0
100                  0
50                   0
4S                   0
6S                   0
TEAM                 0
HS_VALUE             0
HS_NOTOUT            0
BOUNDARY_PCT         0
BOUNDARY_RUNS        0
BOUNDARY_RUNS_PCT    0
dtype: int64
Sav

<span style="font-size: 25px; font-weight: bold;">Combined Ipl_fixtures & Match_results</span> 

In [438]:
import pandas as pd
import os

# Define file paths
fixtures_path = "Ipl Matches 2025/cleaned_ipl_fixtures.csv"
results_path = "Ipl Matches 2025/cleaned_match_results.csv"
output_path = "Ipl Matches 2025/combined_ipl_data_2025.csv"

def combine_ipl_data():
    # Read the CSV files
    try:
        fixtures_df = pd.read_csv(fixtures_path)
        results_df = pd.read_csv(results_path)
        
        print(f"Fixtures data shape: {fixtures_df.shape}")
        print(f"Results data shape: {results_df.shape}")
        
        # Display sample data
        print("\nFixtures sample:")
        print(fixtures_df.head(2))
        print("\nResults sample:")
        print(results_df.head(2))
        
        # Create a common key for merging based on teams and date/time
        fixtures_df['merge_key'] = fixtures_df['Date'] + "_" + fixtures_df['Time'] + "_" + fixtures_df['Team1'] + "_" + fixtures_df['Team2']
        results_df['merge_key'] = results_df['Date'] + "_" + results_df['Time'] + "_" + results_df['Team1'] + "_" + results_df['Team2']
        
        # Perform an outer merge to keep all matches (both with and without results)
        combined_df = pd.merge(fixtures_df, results_df, on='merge_key', how='outer', suffixes=('_fixture', '_result'))
        
        # Clean up the merged dataframe
        # For columns present in both dataframes, keep one version and handle conflicts
        combined_df['Date'] = combined_df['Date_fixture'].fillna(combined_df['Date_result'])
        combined_df['Time'] = combined_df['Time_fixture'].fillna(combined_df['Time_result'])
        combined_df['Team1'] = combined_df['Team1_fixture'].fillna(combined_df['Team1_result'])
        combined_df['Team2'] = combined_df['Team2_fixture'].fillna(combined_df['Team2_result'])
        combined_df['Venue'] = combined_df['Venue_fixture'].fillna(combined_df['Venue_result'])
        
        # Handle Match_No - create sequential match numbers (Match 1, Match 2, etc.)
        combined_df['Match_No'] = ["Match " + str(i+1) for i in range(len(combined_df))]
        
        # Update Status to "Completed" for matches that have results
        combined_df['Status'] = combined_df['Status']
        has_result = combined_df['Winner'].notna()
        combined_df.loc[has_result, 'Status'] = "Completed"
        
        # Fill empty numeric fields with 0
        numeric_columns = ['Team1_Runs', 'Team1_Wickets', 'Team2_Runs', 'Team2_Wickets']
        for column in numeric_columns:
            if column in combined_df.columns:
                combined_df[column] = combined_df[column].fillna(0).astype(int)
        
        # Fill empty over columns with "0.0"
        over_columns = ['Team1_Overs', 'Team2_Overs']
        for column in over_columns:
            if column in combined_df.columns:
                combined_df[column] = combined_df[column].fillna("0.0")
        
        # Fill empty text fields with appropriate placeholders
        combined_df['Winner'] = combined_df['Winner'].fillna("")
        combined_df['Winning_Margin'] = combined_df['Winning_Margin'].fillna("")
        
        # Fill Is_Super_Over with False (or 0) for matches without results
        if 'Is_Super_Over' in combined_df.columns:
            combined_df['Is_Super_Over'] = combined_df['Is_Super_Over'].fillna(False)
        
        # Drop redundant columns
        columns_to_drop = ['Date_fixture', 'Date_result', 'Time_fixture', 'Time_result', 
                           'Team1_fixture', 'Team1_result', 'Team2_fixture', 'Team2_result', 
                           'Venue_fixture', 'Venue_result', 'merge_key']
        
        # Also drop original Match_No columns if they exist with suffixes
        if 'Match_No_fixture' in combined_df.columns:
            columns_to_drop.append('Match_No_fixture')
        if 'Match_No_result' in combined_df.columns:
            columns_to_drop.append('Match_No_result')
            
        combined_df = combined_df.drop(columns=[col for col in columns_to_drop if col in combined_df.columns])
        
        # Reorder columns for better readability
        column_order = ['Match_No', 'Date', 'Time', 'Team1', 'Team2', 'Venue', 'Status',
                        'Team1_Runs', 'Team1_Wickets', 'Team1_Overs', 
                        'Team2_Runs', 'Team2_Wickets', 'Team2_Overs',
                        'Winner', 'Winning_Margin', 'Is_Super_Over']
        
        # Filter to only include columns that exist in the dataframe
        existing_columns = [col for col in column_order if col in combined_df.columns]
        combined_df = combined_df[existing_columns]
        
        # Save the combined data
        combined_df.to_csv(output_path, index=False)
        print(f"\nSuccessfully combined data and saved to {output_path}")
        print(f"Combined data shape: {combined_df.shape}")
        
        # Display sample of Match_No column to verify format
        print("\nMatch_No column sample:")
        print(combined_df['Match_No'].head(5).tolist())
        
        return combined_df
        
    except Exception as e:
        print(f"Error processing files: {e}")
        return None

if __name__ == "__main__":
    # Check if files exist
    if not os.path.exists(fixtures_path):
        print(f"Error: Fixtures file not found at {fixtures_path}")
    elif not os.path.exists(results_path):
        print(f"Error: Results file not found at {results_path}")
    else:
        combined_data = combine_ipl_data()
        
        if combined_data is not None:
            # Display sample of the combined data
            print("\nCombined data sample:")
            print(combined_data.head())
            
            # Count matches by status
            status_counts = combined_data['Status'].value_counts()
            print("\nMatch status summary:")
            print(status_counts)

Fixtures data shape: (32, 8)
Results data shape: (42, 15)

Fixtures sample:
   No  Match_No                  Team1                Team2        Date  \
0   1  Match 43    Chennai Super Kings  Sunrisers Hyderabad  2025-04-25   
1   2  Match 44  Kolkata Knight Riders         Punjab Kings  2025-04-26   

          Time                            Venue     Status  
0  7:30 pm IST  MA Chidambaram Stadium, Chennai  Scheduled  
1  7:30 pm IST            Eden Gardens, Kolkata  Scheduled  

Results sample:
  Match_No        Date      Time                  Team1  \
0  Match 1  2025-03-22  19:30:00  Kolkata Knight Riders   
1  Match 2  2025-03-23  15:30:00    Sunrisers Hyderabad   

                         Team2  Team1_Runs  Team1_Wickets  Team1_Overs  \
0  Royal Challengers Bangalore         174              8         20.0   
1             Rajasthan Royals         286              6         20.0   

   Team2_Runs  Team2_Wickets  Team2_Overs                       Winner  \
0         177          

<span style="font-size: 25px; font-weight: bold;"> Data Cleaning For The Ipl_match_data_43_matches</span> 

In [477]:
import pandas as pd

df = pd.read_csv("Ipl Matches 2025/ipl_match_data_43_matches.csv")

df['TossWinner'] = df['Toss_Details'].str.extract(r'^(.*?)\s+Won')[0].str.strip()
df['TossDecision'] = df['Toss_Details'].str.extract(r'Elected To (Field|Bat)')[0].str.strip()

toss_index = df.columns.get_loc('Toss_Details')

df.drop(columns=['Toss_Details', 'Teams'], inplace=True)  # Removed 'Teams' column
df.insert(toss_index, 'TossWinner', df.pop('TossWinner'))
df.insert(toss_index + 1, 'TossDecision', df.pop('TossDecision'))

df.rename(columns={'MOM': 'Player_of_Match'}, inplace=True)

df['City'] = df['Venue'].str.split(',').str[-1].str.strip()

venue_index = df.columns.get_loc('Venue')

df.drop(columns=['Venue'], inplace=True)
df.insert(venue_index, 'City', df.pop('City'))

df[['Umpire1', 'Umpire2']] = df['On_Field_Umpires'].str.split(',', expand=True)
df['Umpire1'] = df['Umpire1'].str.strip()
df['Umpire2'] = df['Umpire2'].str.strip()

umpire_index = df.columns.get_loc('On_Field_Umpires')

df.drop(columns=['On_Field_Umpires'], inplace=True)

df.insert(umpire_index, 'Umpire2', df.pop('Umpire2'))
df.insert(umpire_index, 'Umpire1', df.pop('Umpire1'))

df['Match_No'] = 'Match ' + df['Match_No'].astype(str)

df.to_csv("Ipl Matches 2025/ipl_match_data_cleaned.csv", index=False)

print("Transformation complete. File saved as 'ipl_match_data_cleaned.csv'")


Transformation complete. File saved as 'ipl_match_data_cleaned.csv'


<span style="font-size: 25px; font-weight: bold;">Preprocessing Combined_ipl_data_2025</span>  

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("Ipl Matches 2025/combined_ipl_data_2025.csv")  # replace with your file path

# Step 1: Replace FALSE with 'No'
df.replace("FALSE", "No", inplace=True)

# Step 2: Split Winning_Margin into WonBy and Margin
df[['Margin', 'WonBy']] = df['Winning_Margin'].str.extract(r'(\d+\.?\d*)\s*(\w+)')
df['Margin'] = df['Margin'].astype(float)  # convert margin to float

# Step 3: Drop original Winning_Margin column
df.drop(columns=['Winning_Margin'], inplace=True)

# Save the cleaned DataFrame to a new CSV file
df.to_csv("Ipl Matches 2025/cleaned_ipl_data_2025.csv", index=False)

# Display the cleaned DataFrame
print(df.head())


<span style="font-size: 25px; font-weight: bold;">Combining</span>  

In [486]:
import pandas as pd

df1 = pd.read_csv("Ipl Matches 2025/ipl_match_data_cleaned.csv")  # Replace with correct path
df2 = pd.read_csv("Ipl Matches 2025/cleaned_ipl_data_2025.csv")  # Replace with correct path

df1['Match_No'] = df1['Match_No'].astype(str)
df2['Match_No'] = df2['Match_No'].astype(str)

df_combined = pd.merge(df1, df2, on="Match_No", how="outer")

df_combined.fillna("N/A", inplace=True)

# Ensure the 'Match_No' column is sorted in ascending order
df_combined['Match_No'] = pd.to_numeric(df_combined['Match_No'].str.extract('(\d+)')[0])
df_combined = df_combined.sort_values(by='Match_No').reset_index(drop=True)

# Convert 'Match_No' back to 'Match X' format
df_combined['Match_No'] = 'Match ' + df_combined['Match_No'].astype(str)

df_combined.to_csv("Ipl Matches 2025/combined_ipl_data_2025.csv", index=False)

print(df_combined.head())


  Match_No           City                   TossWinner TossDecision  \
0  Match 1        Kolkata  Royal Challengers Bengaluru        Field   
1  Match 2      Hyderabad             Rajasthan Royals        Field   
2  Match 3        Chennai          Chennai Super Kings        Field   
3  Match 4  Visakhapatnam               Delhi Capitals        Field   
4  Match 5      Ahmedabad               Gujarat Titans        Field   

                 Umpire1               Umpire2           Third_Umpire  \
0           Vinod Seshan      Abhijeet Bengeri  Madanagopal Jayaraman   
1       Adrian Holdstock          Akshay Totre           Rohan Pandit   
2  Kn Ananthapadmanabhan         Khalid Saiyed            Nitin Menon   
3           Ulhas Gandhe  Abhijit Bhattacharya  Virender Kumar Sharma   
4           Vinod Seshan      Abhijeet Bengeri  Madanagopal Jayaraman   

            Referee                              Player_of_Match        Date  \
0     Prakash Bhatt  Krunal Pandya (Royal Challengers 

<>:14: SyntaxWarning: invalid escape sequence '\d'
<>:14: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ajmal\AppData\Local\Temp\ipykernel_25300\2345879166.py:14: SyntaxWarning: invalid escape sequence '\d'
  df_combined['Match_No'] = pd.to_numeric(df_combined['Match_No'].str.extract('(\d+)')[0])
C:\Users\ajmal\AppData\Local\Temp\ipykernel_25300\2345879166.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_combined.fillna("N/A", inplace=True)


<span style="font-size: 25px; font-weight: bold;">Combined_ipl_data_2025 & team_details</span>  

In [ ]:
import pandas as pd

# Load the datasets
matches_df = pd.read_csv('Ipl Matches 2025/combined_ipl_data_2025.csv')
teams_df = pd.read_csv('Ipl Matches 2025/team_details.csv')

# Merge the datasets on the 'Team' column
combined_df = pd.merge(matches_df, teams_df, how='left', left_on='Team1', right_on='Team')

# Optionally, merge for 'Team2' if needed (you can adjust as per your requirement)
combined_df = pd.merge(combined_df, teams_df, how='left', left_on='Team2', right_on='Team', suffixes=('_Team1', '_Team2'))

# Drop the extra 'Team' column after the merge
combined_df.drop(columns=['Team_Team1', 'Team_Team2'], inplace=True)

# Save the combined data to a new CSV
combined_df.to_csv('Ipl Matches 2025/combined_ipl_data_with_teams.csv', index=False)

# Display the first few rows of the combined dataframe
print(combined_df.head())

In [462]:
print(df.columns)

Index(['Match_No', 'Teams', 'Toss_Details', 'Venue', 'On_Field_Umpires',
       'Third_Umpire', 'Referee', 'MOM'],
      dtype='object')


In [488]:
import pandas as pd
import os

# Define the path
file_path = "Ipl Matches 2025/combined_ipl_data_with_teams.csv"
output_path = "Ipl Matches 2025/Final2025IPL.csv"

# Ensure directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Read the CSV file
df = pd.read_csv(file_path)

# Define the new column order
new_column_order = [
    'Match_No', 'Team1', 'Team2', 'TossWinner', 'TossDecision',
    'Team1_Runs', 'Team1_Wickets', 'Team1_Overs', 'Team2_Runs', 'Team2_Wickets', 'Team2_Overs',
    'Winner', 'Is_Super_Over', 'Margin', 'WonBy', 'Player_of_Match',
    'Umpire1', 'Umpire2', 'Third_Umpire', 'Referee', 'Date', 'Time', 'Venue', 'City', 'Status',
    'Captain_Team1', 'Coach_Team1', 'Owner_Team1', 'Captain_Team2', 'Coach_Team2', 'Owner_Team2'
]

# Clean and standardize 'Is_Super_Over'
df['Is_Super_Over'] = df['Is_Super_Over'].astype(str)
df['Is_Super_Over'] = df['Is_Super_Over'].map(
    lambda x: "No" if x.lower() == "false" else 
              "Yes" if x.lower() == "true" else 
              "NA" if pd.isna(x) or x.strip() == "" else x
)

# Apply logic if Super Over happened
df.loc[df['Is_Super_Over'] == 'Yes', 'Margin'] = 'Super Over'
df.loc[df['Is_Super_Over'] == 'Yes', 'WonBy'] = 'Decided in Super Over'

# Fill other empty cells with "NA"
df = df.fillna("NA")

# Reorder columns
existing_columns = {}
for col in new_column_order:
    if col in df.columns:
        existing_columns[col] = col
    else:
        print(f"Warning: Column '{col}' not found in the dataframe.")

ordered_existing_columns = list(existing_columns.keys())
df_reordered = df[ordered_existing_columns]

# Save the cleaned and updated data
df_reordered.to_csv(output_path, index=False)

print(f"Data has been reorganized and saved to {output_path}")


Data has been reorganized and saved to Ipl Matches 2025/Final2025IPL.csv


C:\Users\ajmal\AppData\Local\Temp\ipykernel_25300\3453993438.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Super Over' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Is_Super_Over'] == 'Yes', 'Margin'] = 'Super Over'
